In [1]:
# python > 3.10 required

In [1]:
!git clone https://github.com/crimson206/openai_proxy_shortcut.git

Cloning into 'openai_proxy_shortcut'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 40 (delta 10), reused 30 (delta 4), pack-reused 0
Receiving objects: 100% (40/40), 238.30 KiB | 9.53 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [26]:
import os

def find_folder_abs_path(foldername):
    for root, dirs, _ in os.walk('.'):
        if foldername in dirs:
            return os.path.abspath(os.path.join(root, foldername))
    return "Folder not found"

ONECLICK_PATH = find_folder_abs_path('oneclick')
print(ONECLICK_PATH)

/content/openai_proxy_shortcut/openai_proxy_shortcut/oneclick


In [2]:
!pip install litellm litellm[proxy];
!pip install ngrok;
!sudo apt-get install screen;
!curl -fsSL https://ollama.com/install.sh | sh;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os

In [66]:
### Google ngrok token
def set_servers(ngrok_auth_token, model_load=True):
  os.environ['NGROK_AUTH_TOKEN'] = ngrok_auth_token
  !screen -S ollama -d -m ollama serve
  if model_load:
    !ollama run llama2 hi.
  config_path = f"{ONECLICK_PATH}/litellm/config.yaml"
  server_path = f"{ONECLICK_PATH}/server.py"
  !screen -S litellm_serve -d -m litellm --config {config_path} --port 8801
  !screen -S remote_serve -d -m python {server_path}

In [5]:
def shutdown_servers():
  !screen -S ollama -X quit
  !screen -S litellm_serve -X quit
  !screen -S remote_serve -X quit

In [6]:
def check_servers():
  print("You must see 3 Socktes")
  !screen -wipe
  with open('server_info', 'r') as file:
    print("\nyour endpoint: ", file.read())
  print("\nmodels available in your endpoint:")
  !curl -X GET "http://0.0.0.0:8801/model/info" -H "accept: application/json"

In [101]:
### Google ngrok token
set_servers(ngrok_auth_token="2cKui3a2XtaD3qsfTnKfjGD1B3P_75PRm9AKo5L41y7qE4Cis", model_load=False)

In [102]:
import time
# Wait for the remote server to be set
time.sleep(10)
check_servers()

You must see 3 Socktes
There are screens on:
	14249.ollama	(02/20/2024 03:38:37 PM)	(Detached)
	14262.remote_serve	(02/20/2024 03:38:37 PM)	(Detached)
	14259.litellm_serve	(02/20/2024 03:38:37 PM)	(Detached)
3 Sockets in /run/screen/S-root.

your endpoint:  https://eaf1-34-91-111-58.ngrok-free.app

models available in your endpoint:
{"data":[{"model_name":"gpt-4","litellm_params":{"model":"ollama/llama2"},"model_info":{"max_tokens":4096,"input_cost_per_token":0.0,"output_cost_per_token":0.0,"litellm_provider":"ollama","mode":"completion"}},{"model_name":"gpt-3.5-turbo","litellm_params":{"model":"ollama/llama2"},"model_info":{"max_tokens":4096,"input_cost_per_token":0.0,"output_cost_per_token":0.0,"litellm_provider":"ollama","mode":"completion"}}]}

In [103]:
### remote server test
import requests

data = {
    "model":"gpt-4",
    "messages":[{ "content": "Hi","role": "user"}],
}

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer OPENAI_API_KEY'
}

response = requests.post(
    "https://eaf1-34-91-111-58.ngrok-free.app/chat/completions",
    headers=headers,
    json=data,
    stream=True,
)

In [104]:
response.json()

{'id': 'chatcmpl-01a96f42-b730-4360-b6e3-2d3315790b45',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': "Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?",
    'role': 'assistant'}}],
 'created': 1708443667,
 'model': 'ollama/llama2',
 'object': 'chat.completion',
 'system_fingerprint': None,
 'usage': {'prompt_tokens': 21, 'completion_tokens': 26, 'total_tokens': 47}}

In [99]:
### If there is problem, shutdown all the sockecs first, and set_servers again.
# shutdown_servers()

No screen session found.
No screen session found.
No screen session found.
